<a href="https://colab.research.google.com/github/mayureshpawashe/chatbot/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community groq sentence-transformers pypdf


In [ ]:
!pip install chromadb

##RAG Reads PDF


In [ ]:
from langchain.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader("sample_data/sample.pdf")
documents = pdf_loader.load()



In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from google.colab import userdata

key = userdata.get('GROQ_API_KEY')


pdf_loader = PyPDFLoader("sample_data/sample.pdf")
documents = pdf_loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(split_docs, embedding_model)

llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=key)

retriever = vector_db.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

while True:
    query = input("\nEnter Your Question (or type 'exit' to quit): ")

    if query.lower() == "exit":
        print("Exiting... 👋")
        break

    response = qa_chain.run(query)
    print("\nAnswer:\n", response)



Enter Your Question (or type 'exit' to quit): exit
Exiting... 👋


#RAG Scans Website

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from google.colab import userdata

key = userdata.get('GROQ_API_KEY')

def scrape_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p")
        text = "\n".join([p.get_text() for p in paragraphs])
        return text
    except Exception as e:
        return f"Error fetching website: {e}"

website_url = input("Enter a website URL to scan: ")

website_text = scrape_website(website_url)

if "Error" in website_text:
    print(website_text)
else:
    print("✅ Website data fetched successfully!")
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = text_splitter.create_documents([website_text])

    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(split_docs, embedding_model)

    llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=key)

    retriever = vector_db.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

    while True:
        query = input("\nEnter Your Question (or type 'exit' to quit): ")

        if query.lower() == "exit":
            print("Exiting... 👋")
            break

        response = qa_chain.run(query)
        print("\nAnswer:\n", response)
